# Hackathon

Some utilities

## Import Utils

In [1]:
!pip install keras
!pip install tables
!pip install scikit-learn



    100% |################################| 337kB 3.3MB/s eta 0:00:01
    100% |################################| 256kB 4.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/2c/f7/79/13f3a12cd723892437c0cfbde1230ab4d82947ff7b3839a4fc
Successfully built pyyaml
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 4.6MB 332kB/s eta 0:00:01
    100% |################################| 399kB 3.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import keras
import h5py as h5
import numpy as np
import pandas as pd
from sklearn import tree

PATH_DATA = 'data/eightieth.h5'
PATH_PREDICT_WITHOUT_GT = 'data/pred_students/pred_eighties_from_half_1_without_gt.h5'
PATH_SUBMIT = 'data/submit/pred_eighties_from_half_1_AWESOMEGROUP.h5'
PATH_PREDICT_WITH_GT = 'data/pred_teachers/pred_eighties_from_half_1.h5'

In [3]:
BATCH_SIZE = 32
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
import keras.layers.normalization 
from keras.callbacks import Callback
%load_ext autoreload
%autoreload 2

In [4]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion): ##La deuxième partie a une taille proportion
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [5]:
def generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
            batch_idxs = sorted(batch_idxs)
            X = f['S2'][batch_idxs, :,:,:]
            Y = f['TOP_LANDCOVER'][batch_idxs, :]
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 23)

In [6]:
idxs = get_idxs(PATH_DATA)
shuffled_idxs = shuffle_idx(idxs)
train_idxs, val_idxs = split_train_val(shuffled_idxs, 0.2)

In [9]:
train_gen = generator(PATH_DATA, BATCH_SIZE, train_idxs)
train_batch_count = get_batch_count(train_idxs, BATCH_SIZE)

val_gen = generator(PATH_DATA, BATCH_SIZE, val_idxs)
val_batch_count = get_batch_count(val_idxs, BATCH_SIZE)

In [10]:
print(train_batch_count, val_batch_count)

5850 1463


In [38]:
dset[:]

array([[12.],
       [12.],
       [ 5.],
       ...,
       [ 2.],
       [ 1.],
       [ 1.]], dtype=float32)

In [ ]:
plt.hist(pd.DataFrame(dset[:]))

KeyboardInterrupt: 

In [18]:
dt_tree = tree.DecisionTreeClassifier()
dt_tree.fit(data4D,dset)

ValueError: Found array with dim 4. Estimator expected <= 2.

# Instanciation du model

In [15]:
input_shape = (16,16,4)
model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))
model.add(Flatten())
model.add(Dense(23))
model.add(Activation('softmax'))

# Fit

In [16]:
# optim = keras.optimizers.Adam(lr=0.001)
optim = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit_generator(train_gen, steps_per_epoch=100, epochs=2, verbose=1, validation_data=val_gen, nb_val_samples=100)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=1, steps_per_epoch=100, validation_steps=100, validation_data=<generator..., epochs=2)`
  


Epoch 1/2
100/100 [==============================] - 8s 82ms/step - loss: 2.5897 - acc: 0.2666 - val_loss: 2.1668 - val_acc: 0.4178
Epoch 2/2
100/100 [==============================] - 7s 71ms/step - loss: 2.1902 - acc: 0.4113 - val_loss: 2.0360 - val_acc: 0.4428


In [31]:
dset = h5.File(PATH_DATA)['TOP_LANDCOVER']
labels = dset[0:]
data4D = h5.File(PATH_DATA)['S2']
print(labels.shape)
print(data.shape)
data = np.zeros((234000,1025))
for i in range(23400):
    
    for l in range(4):
        for j in range(16):
            for k in range(16):
                data[i][(l+1)*(j+1)*(k+1)]=data4D[i][j][k][l]

                print("u = "+str(j*k*l))


(234000, 1)
(234000, 1024)
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u

u = 48
u = 54
u = 60
u = 66
u = 72
u = 78
u = 84
u = 90
u = 0
u = 7
u = 14
u = 21
u = 28
u = 35
u = 42
u = 49
u = 56
u = 63
u = 70
u = 77
u = 84
u = 91
u = 98
u = 105
u = 0
u = 8
u = 16
u = 24
u = 32
u = 40
u = 48
u = 56
u = 64
u = 72
u = 80
u = 88
u = 96
u = 104
u = 112
u = 120
u = 0
u = 9
u = 18
u = 27
u = 36
u = 45
u = 54
u = 63
u = 72
u = 81
u = 90
u = 99
u = 108
u = 117
u = 126
u = 135
u = 0
u = 10
u = 20
u = 30
u = 40
u = 50
u = 60
u = 70
u = 80
u = 90
u = 100
u = 110
u = 120
u = 130
u = 140
u = 150
u = 0
u = 11
u = 22
u = 33
u = 44
u = 55
u = 66
u = 77
u = 88
u = 99
u = 110
u = 121
u = 132
u = 143
u = 154
u = 165
u = 0
u = 12
u = 24
u = 36
u = 48
u = 60
u = 72
u = 84
u = 96
u = 108
u = 120
u = 132
u = 144
u = 156
u = 168
u = 180
u = 0
u = 13
u = 26
u = 39
u = 52
u = 65
u = 78
u = 91
u = 104
u = 117
u = 130
u = 143
u = 156
u = 169
u = 182
u = 195
u = 0
u = 14
u = 28
u = 42
u = 56
u = 70
u = 84
u = 98
u = 112
u = 126
u = 140
u = 154
u = 168
u = 182
u = 196
u = 210
u = 0
u = 15
u =

u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 1
u = 2
u = 3
u = 4
u = 5
u = 6
u = 7
u = 8
u = 9
u = 10
u = 11
u = 12
u = 13
u = 14
u = 15
u = 0
u = 2
u = 4
u = 6
u = 8
u = 10
u = 12
u = 14
u = 16
u = 18
u = 20
u = 22
u = 24
u = 26
u = 28
u = 30
u = 0
u = 3
u = 6
u = 9
u = 12
u = 15
u = 18
u = 21
u = 24
u = 27
u = 30
u = 33
u = 36
u = 39
u = 42
u = 45
u = 0
u = 4
u = 8
u = 12
u = 16
u = 20
u = 24
u = 28
u = 32
u = 36
u = 40
u = 44
u = 48
u = 52
u = 56
u = 60
u = 0
u = 5
u = 10
u = 15
u = 20
u = 25
u = 30
u = 35
u = 40
u = 45
u = 50
u = 55
u = 60
u = 65
u = 70
u = 75
u = 0
u = 6
u = 12
u = 18
u = 24
u = 30
u = 36
u = 42
u = 48
u = 54
u = 60
u = 66
u = 72
u = 78
u = 84
u = 90
u = 0
u = 7
u = 14
u = 21
u = 28
u = 35
u = 42
u = 49
u = 56
u = 63
u = 70
u = 77
u = 84
u = 91
u = 98
u = 105
u = 0
u = 8
u = 16
u = 24
u = 32
u = 40
u = 48
u = 56
u = 64
u = 72
u = 80
u = 88
u = 96
u = 104
u = 112
u = 120
u = 0
u = 9
u = 

u = 3
u = 6
u = 9
u = 12
u = 15
u = 18
u = 21
u = 24
u = 27
u = 30
u = 33
u = 36
u = 39
u = 42
u = 45
u = 0
u = 6
u = 12
u = 18
u = 24
u = 30
u = 36
u = 42
u = 48
u = 54
u = 60
u = 66
u = 72
u = 78
u = 84
u = 90
u = 0
u = 9
u = 18
u = 27
u = 36
u = 45
u = 54
u = 63
u = 72
u = 81
u = 90
u = 99
u = 108
u = 117
u = 126
u = 135
u = 0
u = 12
u = 24
u = 36
u = 48
u = 60
u = 72
u = 84
u = 96
u = 108
u = 120
u = 132
u = 144
u = 156
u = 168
u = 180
u = 0
u = 15
u = 30
u = 45
u = 60
u = 75
u = 90
u = 105
u = 120
u = 135
u = 150
u = 165
u = 180
u = 195
u = 210
u = 225
u = 0
u = 18
u = 36
u = 54
u = 72
u = 90
u = 108
u = 126
u = 144
u = 162
u = 180
u = 198
u = 216
u = 234
u = 252
u = 270
u = 0
u = 21
u = 42
u = 63
u = 84
u = 105
u = 126
u = 147
u = 168
u = 189
u = 210
u = 231
u = 252
u = 273
u = 294
u = 315
u = 0
u = 24
u = 48
u = 72
u = 96
u = 120
u = 144
u = 168
u = 192
u = 216
u = 240
u = 264
u = 288
u = 312
u = 336
u = 360
u = 0
u = 27
u = 54
u = 81
u = 108
u = 135
u = 162
u = 189
u = 216
u = 

u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 

...

u = 180
u = 0
u = 13
u = 26
u = 39
u = 52
u = 65
u = 78
u = 91
u = 104
u = 117
u = 130
u = 143
u = 156
u = 169
u = 182
u = 195
u = 0
u = 14
u = 28
u = 42
u = 56
u = 70
u = 84
u = 98
u = 112
u = 126
u = 140
u = 154
u = 168
u = 182
u = 196
u = 210
u = 0
u = 15
u = 30
u = 45
u = 60
u = 75
u = 90
u = 105
u = 120
u = 135
u = 150
u = 165
u = 180
u = 195
u = 210
u = 225
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 2
u = 4
u = 6
u = 8
u = 10
u = 12
u = 14
u = 16
u = 18
u = 20
u = 22
u = 24
u = 26
u = 28
u = 30
u = 0
u = 4
u = 8
u = 12
u = 16
u = 20
u = 24
u = 28
u = 32
u = 36
u = 40
u = 44
u = 48
u = 52
u = 56
u = 60
u = 0
u = 6
u = 12
u = 18
u = 24
u = 30
u = 36
u = 42
u = 48
u = 54
u = 60
u = 66
u = 72
u = 78
u = 84
u = 90
u = 0
u = 8
u = 16
u = 24
u = 32
u = 40
u = 48
u = 56
u = 64
u = 72
u = 80
u = 88
u = 96
u = 104
u = 112
u = 120
u = 0
u = 10
u = 20
u = 30
u = 40
u = 50
u = 60
u = 70
u = 80
u = 90
u = 100
u = 110
u = 120
u = 13

u = 240
u = 0
u = 18
u = 36
u = 54
u = 72
u = 90
u = 108
u = 126
u = 144
u = 162
u = 180
u = 198
u = 216
u = 234
u = 252
u = 270
u = 0
u = 20
u = 40
u = 60
u = 80
u = 100
u = 120
u = 140
u = 160
u = 180
u = 200
u = 220
u = 240
u = 260
u = 280
u = 300
u = 0
u = 22
u = 44
u = 66
u = 88
u = 110
u = 132
u = 154
u = 176
u = 198
u = 220
u = 242
u = 264
u = 286
u = 308
u = 330
u = 0
u = 24
u = 48
u = 72
u = 96
u = 120
u = 144
u = 168
u = 192
u = 216
u = 240
u = 264
u = 288
u = 312
u = 336
u = 360
u = 0
u = 26
u = 52
u = 78
u = 104
u = 130
u = 156
u = 182
u = 208
u = 234
u = 260
u = 286
u = 312
u = 338
u = 364
u = 390
u = 0
u = 28
u = 56
u = 84
u = 112
u = 140
u = 168
u = 196
u = 224
u = 252
u = 280
u = 308
u = 336
u = 364
u = 392
u = 420
u = 0
u = 30
u = 60
u = 90
u = 120
u = 150
u = 180
u = 210
u = 240
u = 270
u = 300
u = 330
u = 360
u = 390
u = 420
u = 450
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 3
u = 6
u = 9
u = 12
u = 15
u 

u = 72
u = 90
u = 108
u = 126
u = 144
u = 162
u = 180
u = 198
u = 216
u = 234
u = 252
u = 270
u = 0
u = 21
u = 42
u = 63
u = 84
u = 105
u = 126
u = 147
u = 168
u = 189
u = 210
u = 231
u = 252
u = 273
u = 294
u = 315
u = 0
u = 24
u = 48
u = 72
u = 96
u = 120
u = 144
u = 168
u = 192
u = 216
u = 240
u = 264
u = 288
u = 312
u = 336
u = 360
u = 0
u = 27
u = 54
u = 81
u = 108
u = 135
u = 162
u = 189
u = 216
u = 243
u = 270
u = 297
u = 324
u = 351
u = 378
u = 405
u = 0
u = 30
u = 60
u = 90
u = 120
u = 150
u = 180
u = 210
u = 240
u = 270
u = 300
u = 330
u = 360
u = 390
u = 420
u = 450
u = 0
u = 33
u = 66
u = 99
u = 132
u = 165
u = 198
u = 231
u = 264
u = 297
u = 330
u = 363
u = 396
u = 429
u = 462
u = 495
u = 0
u = 36
u = 72
u = 108
u = 144
u = 180
u = 216
u = 252
u = 288
u = 324
u = 360
u = 396
u = 432
u = 468
u = 504
u = 540
u = 0
u = 39
u = 78
u = 117
u = 156
u = 195
u = 234
u = 273
u = 312
u = 351
u = 390
u = 429
u = 468
u = 507
u = 546
u = 585
u = 0
u = 42
u = 84
u = 126
u = 168
u = 210
u

u = 180
u = 195
u = 210
u = 225
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 0
u = 2
u = 4
u = 6
u = 8
u = 10
u = 12
u = 14
u = 16
u = 18
u = 20
u = 22
u = 24
u = 26
u = 28
u = 30
u = 0
u = 4
u = 8
u = 12
u = 16
u = 20
u = 24
u = 28
u = 32
u = 36
u = 40
u = 44
u = 48
u = 52
u = 56
u = 60
u = 0
u = 6
u = 12
u = 18
u = 24
u = 30
u = 36
u = 42
u = 48
u = 54
u = 60
u = 66
u = 72
u = 78
u = 84
u = 90
u = 0
u = 8
u = 16
u = 24
u = 32
u = 40
u = 48
u = 56
u = 64
u = 72
u = 80
u = 88
u = 96
u = 104
u = 112
u = 120
u = 0
u = 10
u = 20
u = 30
u = 40
u = 50
u = 60
u = 70
u = 80
u = 90
u = 100
u = 110
u = 120
u = 130
u = 140
u = 150
u = 0
u = 12
u = 24
u = 36
u = 48
u = 60
u = 72
u = 84
u = 96
u = 108
u = 120
u = 132
u = 144
u = 156
u = 168
u = 180
u = 0
u = 14
u = 28
u = 42
u = 56
u = 70
u = 84
u = 98
u = 112
u = 126
u = 140
u = 154
u = 168
u = 182
u = 196
u = 210
u = 0
u = 16
u = 32
u = 48
u = 64
u = 80
u = 96
u = 112
u = 128
u = 144
u = 160

KeyboardInterrupt: 

In [16]:
data=np.zeros((3,2))

data[2][1]

0.0

## Prediction routines

In order to submit a result here are some gits

In [27]:
import os 
def prediction_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        X = f['S2'][batch_idxs, :,:,:]
        yield np.array(X)

def build_h5_pred_file(pred, h5_output_path):
    if os.path.exists(h5_output_path):
        os.remove(h5_output_path)
    f = h5.File(h5_output_path, 'w')
    top_landcover_submit = f.create_dataset("TOP_LANDCOVER", (len(pred), 1), maxshape=(None, 1))
    top_landcover_submit[:, 0] = pred
    f.close()
    
    return 1

In [16]:
pred_idx = get_idxs(PATH_PREDICT_WITHOUT_GT)
print(len(pred_idx))
pred_gen = prediction_generator(PATH_PREDICT_WITHOUT_GT, BATCH_SIZE, pred_idx)
prediction = model.predict_generator(pred_gen, steps=get_batch_count(pred_idx, BATCH_SIZE), verbose=1)
print(len(prediction))
build_h5_pred_file(np.argmax(prediction, axis = 1), PATH_SUBMIT)

241700
7554/7554 [==============================] - 17s 2ms/step
241700


1

## Some ideas for monitoring

In [ ]:
def gt_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        Y = f['TOP_LANDCOVER'][batch_idxs, :]
        yield keras.utils.np_utils.to_categorical(np.array(Y), 23)

gt_gen = gt_generator(PATH_PREDICT_WITH_GT, BATCH_SIZE, pred_idx)
gt = []
for elem in gt_gen:
    gt.append(elem)
gt = np.vstack(gt)

In [41]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",fontsize=7)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def clean_confusion_matrix(confusion_matrix, classes):
    real_classes = []
    for c in range(len(classes)):
        if np.sum(confusion_matrix[:,c])+np.sum(confusion_matrix[c, :]) != 0:
            real_classes.append(c)
    real_confusion_matrix = np.empty((len(real_classes), len(real_classes)))  
    for c_index in range(len(real_classes)):
        real_confusion_matrix[c_index,:] = confusion_matrix[real_classes[c_index], real_classes]
    return real_confusion_matrix, real_classes

In [ ]:
%matplotlib notebook
from sklearn.metrics import confusion_matrix
y_true = np.argmax(gt, axis=1)
y_pred = np.argmax(prediction, axis = 1)

real_cnf_matrix, real_classes = clean_confusion_matrix(confusion_matrix(y_true, y_pred, labels= range(23)), range(23))
plot_confusion_matrix(real_cnf_matrix, classes = real_classes, normalize=True)